<a href="https://colab.research.google.com/github/adrnabrm/beachhacks-spring25/blob/main/finetuning_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [9]:
# Installing unsloth
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [10]:
# LoRA Configuration
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.3.18 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [13]:
alpaca_prompt= """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{ingredients}

### Output:
{recipe}"""

EOS_TOKEN = tokenizer.eos_token

In [18]:
import json
from datasets import Dataset

# Loads the dataset as a Dataset object (using a list of dictionaries, for example)
with open('dataset.json', 'r') as f:
    data = json.load(f)

# Converts it to a Hugging Face Dataset
dataset = Dataset.from_list(data)

# Define the formatting function
def formatting_prompts_func(examples):
    instructions = examples["system"]
    ingredients = examples["input"]
    recipe = examples["output"]
    texts = []
    for instruction, ingredients, recipe in zip(instructions, ingredients, recipe):
        text = alpaca_prompt.format(instruction=instruction, ingredients=ingredients, recipe=recipe) + EOS_TOKEN
        texts.append(text)
    return { "text": texts }

# Apply the formatting function to the dataset
dataset = dataset.map(formatting_prompts_func, batched=True)

# Output the formatted dataset
print(dataset[0]['system'])  # To print the first formatted entry


Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

You are an AI chef. Given a list of ingredients, generate a recipe in the following JSON format that will always have a title, ingredients WITH specified measurements, and detailed instructions:
  ```json
{
  "title": "Recipe title",
  "ingredients": ["ingredient1", "ingredient2", ...],
  "directions": ["step1", "step2", ...]
}

